In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
# from data.py
def get_data (csv_file):
    df = pd.read_csv('../raw_data/' + f'{csv_file}')
    return df

In [24]:
#from matrix.py ######################### OLD VERSION ###########################################

def encode_categories_articles(clusters_articles_df):  #df doit avoir les champs format ['article_id','catID']
    # création de dummies pour chaque catégorie de produits
    cluster_name=clusters_articles_df.columns[1]
    ohe = OneHotEncoder(sparse=False)# Instanciate encoder
    cat_encoded=pd.DataFrame(ohe.fit_transform(clusters_articles_df[[cluster_name]]))
    cat_names=ohe.categories_
    return pd.concat([articles_hetm[['article_id']],cat_encoded], axis=1) 


def build_segment_base(segment): # ce df contient toutes les transactions des individus du segment 
    return segment[['customer_id']].merge(transact_history)


def build_matrix(segment_base,X_week,horizon):
    #filtrage sur l'horizon temporel pour connaitre les préférences des customers
    week_min= X_week-horizon+1
    week_max= X_week
    filtre=np.logical_and(segment_base['t_dat'] >= week_min,segment_base['t_dat'] <= week_max)

    Xh_week_achats=segment_base[filtre][['customer_id','article_id']]
    
    Xh_week_achats['counter']=1
    Xh_week_achats=Xh_week_achats.merge(articles_categories)
   
    # je fais un groupby par clients, sans le sort car j'ai ensuite besoin de refaire un tri 
    # donc gain de temps de calcul car on suppose que sort_values est plus rapide que le sort de groupby
    achats_gb_cust_cat=Xh_week_achats.groupby(['customer_id'],as_index=False,sort=False)
    achats_gb_cust_cat_srt=achats_gb_cust_cat.sum().sort_values(by=['customer_id','counter'],ascending=False)
    
    # je refais ensuite un groupby par client pour calculer la somme de chaque dummy de categories
    segment_matrix=achats_gb_cust_cat_srt.drop(columns=['article_id']).groupby(['customer_id'],as_index=False).sum()
    return segment_matrix

# from matrix.py (suite)

def get_max_cats(segment_matrix, diversity):
    # diversity est compris entre 1 et 12 au maximum puisque nous devons restituer 12 articles_id comme prédiction
    # idéalement maximum 4 pour chercher ensuite le Top des articles vendus dans cheaque catégorie considérée
    # Si 1 on s'intéresse uniquement à la catégorie la plus férquemment acehtée sur l'horizon définit au préalable,
    # 2 ce sera les 2 les plus importantes, etc...

    if diversity==1:
        
        # slice des colonnes pour ne tenir comptes que des colonnes des dummies catégories
        df=segment_matrix.drop(columns=['customer_id','counter']) #les deux premières colonnes sont cust_id et counter
        # je calcule le max
        cat_max1=pd.DataFrame(df.max(axis=1),columns=['cat_max1']) # je cherche le max sur les lignes à priori inutile car
        #on a besoin de la categorie max peu importe les occurrences
        id_cat_max1=pd.DataFrame(df.idxmax(axis=1),columns=['cat_max1'])
               
        cust_max_cats=pd.concat([segment_matrix['customer_id'],id_cat_max1], axis=1) 
        
    return cust_max_cats


In [52]:
#from matrix.py ######################### NEW VERSION ###########################################

def encode_categories_articles(clusters_articles_df):  #df doit avoir les champs format ['article_id','catID']
    # création de dummies pour chaque catégorie de produits
    cluster_name=clusters_articles_df.columns[1]
    ohe = OneHotEncoder(sparse=False)# Instanciate encoder
    cat_encoded=pd.DataFrame(ohe.fit_transform(clusters_articles_df[[cluster_name]]))
    cat_names=ohe.categories_
    return pd.concat([articles_hetm[['article_id']],cat_encoded], axis=1) 


def build_segment_base(segment): # ce df contient toutes les transactions des individus du segment 
    return segment[['customer_id']].merge(transact_history)


def build_segment_horizon_transactions(segment_base,X_week,horizon):
    #filtrage sur l'horizon temporel pour connaitre les préférences des customers    
    week_min= X_week-horizon+1
    week_max= X_week
    filtre=np.logical_and(segment_base['t_dat'] >= week_min,segment_base['t_dat'] <= week_max)
    Xh_week_achats=segment_base[filtre][['customer_id','article_id']]
    Xh_week_achats['counter']=1
    return Xh_week_achats

def build_matrix(Xh_week_achats):
    achats_for_matrix=Xh_week_achats
    #achats_for_matrix['counter']=1 je pense pouvoir supprimer
    achats_for_matrix=achats_for_matrix.merge(articles_categories)

    # je fais un groupby par clients, sans le sort car j'ai ensuite besoin de refaire un tri
    # donc gain de temps de calcul car on suppose que sort_values est plus rapide que le sort de groupby
    achats_gb_cust_cat=achats_for_matrix.groupby(['customer_id'],as_index=False,sort=False)
    achats_gb_cust_cat_srt=achats_gb_cust_cat.sum().sort_values(by=['customer_id','counter'],ascending=False)

    # je refais ensuite un groupby par client pour calculer la somme de chaque dummy de categories
    segment_matrix=achats_gb_cust_cat_srt.drop(columns=['article_id']).groupby(['customer_id'],as_index=False).sum()
    return segment_matrix

def get_max_cats(Xh_week_achats,clusters_articles_df):
    # contrairement à la construction de la matrice, ici je veut les infos de transactions en lignes
    tmp=Xh_week_achats.merge(clusters_articles_df)
    cluster_name=clusters_articles_df.columns[1]
    tmp=tmp.groupby(['customer_id',cluster_name],as_index=False,sort=False).sum()
    achats_for_max_cats=tmp.sort_values(by=['customer_id','counter'],ascending=False)
    
    return achats_for_max_cats



In [296]:
#from baseline0.py
#Baseline_0(S1,article_history,103,104,1)
def Baseline_0(segment,transact_history,X_week,y_week,horizon): # exemple: week104name ='in104' dans l'historique des transactions

    X_week_dummy="in" + f'{X_week}'
    y_week_dummy="in" + f'{y_week}'

    filtre_Xweek=transact_history[X_week_dummy]==1
    #filtre_yweek=transact_history[y_week_dummy]==1

    X_achats=segment[['customer_id']].merge(transact_history[filtre_Xweek][['customer_id','article_id',X_week_dummy]])
    # pour précision : y_achats=segment[['customer_id']].merge(transact_history[filtre_yweek][['customer_id','article_id',y_week_dummy]])

    # calcul des 12 plus populaires du segment:
    articles= X_achats.groupby(['article_id']).sum()
    Top12_arr=articles.sort_values(by=X_week_dummy, ascending=False).head(12)
    Top12=""
    for i in range (0, 12):
        if i==0 :
            Top12=str(Top12_arr.index[i])
        else:
            Top12=Top12 + " "+ str(Top12_arr.index[i])

    #je donne à tous mes customers de mon segment le baseline top12 calculé ci-dessus
    segment['Baseline0']=Top12

    #segment_BS=segment.merge(Baseline0_precision, how = 'left', on='customer_id')

    return segment


In [5]:
# Récupértaion de segments clients de Naceur selon l'âge
# note du 06042022: à ce stade de l'analyse les customers sans information sur l'âge ne sont pas inclus
#get_data('customers_age_16_17.csv')
#get_data('customers_age_18_19.csv','S2')
#get_data('customers_age_20_21.csv','S3')
#get_data('customers_age_22_23.csv','S4')
#get_data('customers_age_24_25.csv','S5')
#get_data('customers_age_26_27.csv','S6')
#get_data('customers_age_28_29.csv','S7')
#get_data('customers_age_30_34.csv','S8')
#get_data('customers_age_35_39.csv','S9')
#get_data('customers_age_40_44.csv','S10')
#set_data('customers_age_45_49.csv','S11')
#get_data('customers_age_50_54.csv','S12')
#get_data('customers_age_55_59.csv','S13')
#get_data('customers_age_60.csv','S14')
S1=get_data('customers_age_16_17.csv')

In [6]:
# Récupération du petit fichier de transactions "test" contenant semaines 101 à 104:
transactions_file='transaction_train_week_101_104.csv'
transactions_file_path=f'../raw_data/{transactions_file}'
transact_history=get_data(transactions_file_path)

In [7]:
#cleaning du petit fichier utilisé pour tester les programmes
transactions_101_104 = transact_history
transactions_101_104.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'price', 'sales_channel_id'], inplace=True)

In [8]:
#construction de la base_segment utile pour le calcul des valeurs des éléments de la matrice
S1_base=build_segment_base(S1)

,customer_id,t_dat,article_id
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998001
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998009
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,101,926502001
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,101,501620049
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,101,885077003


In [31]:
S1_base

,customer_id,t_dat,article_id
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998001
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998009
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,101,926502001
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,101,501620049
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,101,885077003
...,...,...,...
5950,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,104,914441006
5951,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,104,914441004
5952,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,104,858147006
5953,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,104,892643001


In [9]:
#création des clusters d'articles
articles_hetm = get_data('../raw_data/articles.csv')
articles_hetm=articles_hetm[['article_id','garment_group_name']] #fait office de clusters_articles_df
articles_categories=encode_categories_articles(articles_hetm)

In [10]:
articles_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   article_id  105542 non-null  int64  
 1   0           105542 non-null  float64
 2   1           105542 non-null  float64
 3   2           105542 non-null  float64
 4   3           105542 non-null  float64
 5   4           105542 non-null  float64
 6   5           105542 non-null  float64
 7   6           105542 non-null  float64
 8   7           105542 non-null  float64
 9   8           105542 non-null  float64
 10  9           105542 non-null  float64
 11  10          105542 non-null  float64
 12  11          105542 non-null  float64
 13  12          105542 non-null  float64
 14  13          105542 non-null  float64
 15  14          105542 non-null  float64
 16  15          105542 non-null  float64
 17  16          105542 non-null  float64
 18  17          105542 non-null  float64
 19  18

In [11]:
articles_categories.head()

,article_id,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,108775015,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108775044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,108775051,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110065001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,110065002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [42]:
S1_Xh_week_achats=build_segment_horizon_transactions(S1_base,103,1)

In [44]:
S1_Xh_week_achats.shape

(885, 3)

In [49]:
S1_Xh_week_achats

,customer_id,article_id,counter
18,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388003,1
19,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388018,1
20,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388001,1
21,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388001,1
22,00953d607bb9b498cf020c314436b74c54876e1f03739a...,866731001,1
...,...,...,...
5866,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,853881001,1
5871,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,621381014,1
5899,fe452b72e97790e68dcf79f49521782d486c345981f773...,487800051,1
5907,feb01a2d17ca601bb4361099013b9010773778a3da3552...,706016003,1


In [45]:
build_matrix(S1_Xh_week_achats)

,customer_id,counter,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,5,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,017e589a1b5990b94288741a86252e14c1c361f3802827...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
340,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
341,fe452b72e97790e68dcf79f49521782d486c345981f773...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,feb01a2d17ca601bb4361099013b9010773778a3da3552...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [54]:
achats_for_max_cats=get_max_cats(S1_Xh_week_achats,articles_hetm)

In [58]:
achats_for_max_cats.head()

,customer_id,garment_group_name,article_id,counter
601,fede72be6d8a2130624098f344c15f4d88a45d8b3cbbc9...,Accessories,896123001,1
101,feb01a2d17ca601bb4361099013b9010773778a3da3552...,Trousers,706016003,1
600,fe452b72e97790e68dcf79f49521782d486c345981f773...,Jersey Basic,487800051,1
151,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,Trousers,621381014,1
598,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,Swimwear,559633008,1


In [57]:
achats_for_max_cats[achats_for_max_cats['customer_id']=='0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18f0ca611d4938ab1a48']

,customer_id,garment_group_name,article_id,counter
19,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Trousers,1791672006,2
21,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Jersey Basic,1431248020,2


In [ ]:
################## OLD WAY TO BUILD DATA #######################################

In [12]:
#construction de ma matrice par segment
S1_matrix=build_matrix(S1_base,103,1)

In [13]:
S1_matrix.head(10)

,customer_id,counter,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,5,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,017e589a1b5990b94288741a86252e14c1c361f3802827...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
5,03657a8037463a757d6e61f74092a8a16780b2b17d78c0...,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,039b7f5c244ed9d0481616cc8a8a4947ae6103ef457186...,3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,04614f1e945c9d122409d402c69e4f7531b163a9411d5d...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0534641cdec5e38b903a9d4f34fb6a13f1545f923a2884...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,05570e4941fbf752a1356da4e0d64a723c7de01abedc4d...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df,catmax1,id_cat_max1,cust_max_cats=get_max_cats(S1_matrix, 1)

In [15]:
df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
341,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
catmax1

,cat_max1
0,4.0
1,1.0
2,1.0
3,1.0
4,2.0
...,...
339,1.0
340,1.0
341,1.0
342,1.0


In [17]:
id_cat_max1

,cat_max1
0,5
1,5
2,0
3,3
4,5
...,...
339,6
340,16
341,5
342,16


In [18]:
cust_max_cats

,customer_id,cat_max1
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,5
1,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,5
2,017e589a1b5990b94288741a86252e14c1c361f3802827...,0
3,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,3
4,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,5
...,...,...
339,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,6
340,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,16
341,fe452b72e97790e68dcf79f49521782d486c345981f773...,5
342,feb01a2d17ca601bb4361099013b9010773778a3da3552...,16


In [27]:
verif,matrix=build_matrix(S1_base,103,1)

In [28]:
verif

,customer_id,article_id,counter,0,1,2,3,4,5,6,...,11,12,13,14,15,16,17,18,19,20
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388003,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388018,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8cdf55b072cfc37c6430ff3b87b65998cf851dbfc746a6...,806388018,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,e5c116cd5c54ff40714246da3c81ac926725635f71e476...,806388018,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00953d607bb9b498cf020c314436b74c54876e1f03739a...,806388001,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,fd0d0860c52aea5a8b38b01550603e88fe5a6008e19eca...,756318001,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
881,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,559633008,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
882,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,853881001,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
883,fe452b72e97790e68dcf79f49521782d486c345981f773...,487800051,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
############################### NEW VERSION ######################################